<a href="https://colab.research.google.com/github/nipetrov/Homework-ML/blob/master/MLatFIAN2020_seminar04_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Task 1

(Titanic data again)

Build a model with `sklearn`'s `LogisticRegression` or `SVC` to get the accuracy of at least 0.81 on the test set. Can you get higher? 0.84?

Some (optional) suggestions:
- Add new features (e.g. missing value indicator columns)
- Fill missing values
- Encode categorical features (e.g. one-hot encoding)
- Scale the features (e.g. with standard or robust scaler)
- Think of other ways of preprocessing the features (e.g. `Fare` $\to$ `log(Fare)`)
- Try adding polynomial features
- use `sklearn.model_selection.GridSearchCV` to search for the best hyperparameters (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)



In [ ]:
!wget https://github.com/HSE-LAMBDA/MLDM-2020/raw/master/day-01/train.csv

--2020-10-12 19:40:59--  https://github.com/HSE-LAMBDA/MLDM-2020/raw/master/day-01/train.csv
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/HSE-LAMBDA/MLDM-2020/master/day-01/train.csv [following]
--2020-10-12 19:41:00--  https://raw.githubusercontent.com/HSE-LAMBDA/MLDM-2020/master/day-01/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>]  58.89K  --.-KB/s    in 0.02s   

2020-10-12 19:41:00 (2.33 MB/s) - ‘train.csv.1’ saved [60302/60302]



In [ ]:
import pandas as pd
data = pd.read_csv("train.csv", index_col='PassengerId')
data.head()
#dataset = data.copy()
#a = []
#male = -1 
#for i in range(len(dataset)):
#  if dataset["Sex"].iloc[i] == "female" :
#    male = 1
#  else : 
#    male = 2
#  a.append(dataset.groupby(['Sex', 'Pclass']).Age.median()[dataset.Pclass.iloc[i]*male - 1])
#npa = np.array(a)
#dataset.Age = np.where(dataset.Age.isna(), npa, dataset.Age )
#plt.hist(dataset.Age, bins = 200)
#dataset.groupby(['Sex', 'Pclass']).Age.median()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### About the data
Here's some of the columns
* Name - a string with person's full name
* Survived - 1 if a person survived the shipwreck, 0 otherwise.
* Pclass - passenger class. Pclass == 3 is cheap'n'cheerful, Pclass == 1 is for moneybags.
* Sex - a person's gender
* Age - age in years, if available
* SibSp - number of siblings on a ship
* Parch - number of parents on a ship
* Fare - ticket cost
* Embarked - port where the passenger embarked
 * C = Cherbourg; Q = Queenstown; S = Southampton

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

def feature_selection_and_preprocessing(dataset):
  a = []
  male = -1 
  for i in range(len(dataset)):
    if dataset["Sex"].iloc[i] == "female" :
      male = 1
    else : 
      male = 2
    a.append(dataset.groupby(['Sex', 'Pclass']).Age.median()[dataset.Pclass.iloc[i]*male - 1])
  npa = np.array(a)
  features = dataset[["Fare"]].copy()
  features.Fare = np.log(np.where(features.Fare > 11., features.Fare, 1))
  features["Family"] = dataset.SibSp + 5*dataset.Parch 
  features["Sex"] = dataset.Sex
  features["Pclass"] = dataset.Pclass
  features["Age"] = dataset.Age 
  features.Age = np.where(features.Age.isna(), npa, features.Age )
#  features["Age"] = dataset.Age.fillna(dataset.Age.median())
  features.Age -= features.Age.median()
  features.Age /= features.Age.std()
  features['Embarked'] = data.Embarked.fillna('S')

  return features

model = make_pipeline(
    # <YOUR CODE>
    # E.g.
    make_column_transformer(
        (OneHotEncoder(sparse=False), ['Embarked', 'Sex']),
        remainder='passthrough'
    ),
    RobustScaler(),
    PolynomialFeatures(degree = 3),
#    SVC(C = 0.001, kernel = 'linear')
    LogisticRegression(max_iter= 700, C = 0.03)
)


# Validation code (do not touch)
data = pd.read_csv("train.csv", index_col='PassengerId')
data_train, data_test = train_test_split(data, test_size=200, random_state=42)

model.fit(
    feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    ),
    data_train['Survived']
)

train_predictions = model.predict(
    feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    )
)

test_predictions = model.predict(
    feature_selection_and_preprocessing(
        data_test.drop('Survived', axis=1)
    )
)

print("Train accuracy:", accuracy_score(
    data_train['Survived'],
    train_predictions
))
print("Test accuracy:", accuracy_score(
    data_test['Survived'],
    test_predictions
))

Train accuracy: 0.8422575976845152
Test accuracy: 0.835
